In [1]:
import pandas as pd
import json
import datetime
import time
import numpy as np
import cv2
import matplotlib.pyplot as plt
from ipywidgets import (interact, interactive, Select, IntSlider, SelectMultiple, Layout, VBox, HBox, fixed, Output ,
                        interactive_output,Dropdown, RadioButtons, Tab, Label, Button, Text)
import matplotlib.image as mpimg
import glob

In [6]:
df_all = pd.concat(map(pd.read_csv, glob.glob('../WEB_API/DB_加工済みDF/*.csv'))) 
df_all['o\'clock'] = df_all['o\'clock'].astype('str').str.zfill(2)
df_all['timestamp_min_S'] = pd.to_datetime(df_all['timestamp_min_S'])
df_all['timestamp_max_S'] = pd.to_datetime(df_all['timestamp_max_S'])
df_all['timestamp_min_E'] = pd.to_datetime(df_all['timestamp_min_E'])
df_all['timestamp_max_E'] = pd.to_datetime(df_all['timestamp_max_E'])
df_all['day_of_week'] = df_all['timestamp_min_S'].dt.strftime('%a')

In [11]:
#ver1.1
def interactive_path_graph():
    
    # 各ウィジェット用の変数
    area_list = [None, '本町3丁目', '本町4丁目', '行神橋', '市役所側']
    style = {'description_width': 'initial'}
    direction_dict = {'本町3丁目～本町4丁目':'A1_to_A2', '本町3丁目～行神橋':'A1_to_A3', '本町3丁目～市役所側':'A1_to_A4',
                      '本町3丁目～他':'A1_to_other', '本町4丁目～本町3丁目':'A2_to_A1', '本町4丁目～行神橋':'A2_to_A3',
                      '本町4丁目～市役所側':'A2_to_A4', '本町4丁目～他':'A2_to_other', '行神橋～本町3丁目':'A3_to_A1',
                      '行神橋～本町4丁目':'A3_to_A2', '行神橋～市役所側':'A3_to_A4', '行神橋～他':'A3_to_other',
                      '市役所側～本町3丁目':'A4_to_A1', '市役所側～本町4丁目':'A4_to_A2', '市役所側～行神橋':'A4_to_A3', 
                      '市役所側～他':'A4_to_other'}
    
    # メイングラフ用のWidgets
    w5 = SelectMultiple(description='経路:', options=direction_dict.keys(), layout=Layout(height='250px'))
    w6 = SelectMultiple(description='午前午後:', options=['午前', '午後'], layout=Layout(height='50px'))
    w8 = SelectMultiple(description='時刻:', options=np.sort(df_all['o\'clock'].str.zfill(2).unique()) )
    w9 = SelectMultiple(description='曜日:', options=df_all['day_of_week'].unique() )
    w10 = Dropdown(description='始点:', options=area_list)
    w11 = Dropdown(description='終点:', options=area_list)
    w12 = SelectMultiple(description='日付:', options=df_all['timestamp_min_S'].dt.strftime('%Y-%m%-%d %H:%M:%S.%f').str[:10].unique(),
                         layout=Layout(height='500px'))
    w13 = RadioButtons(description='Sum_or_Mean', options=['sum', 'mean'], disabled=False)

    #サブグラフ用のWidgets   ※ mw = mirror_widgets
    mw5 = SelectMultiple(description='経路:', options=direction_dict.keys(), layout=Layout(height='250px'))
    mw6 = SelectMultiple(description='午前午後:', options=['午前', '午後'], layout=Layout(height='50px'))
    mw8 = SelectMultiple(description='時刻:', options=np.sort(df_all['o\'clock'].str.zfill(2).unique()) )
    mw9 = SelectMultiple(description='曜日:', options=df_all['day_of_week'].unique() )
    mw10 = Dropdown(description='始点:', options=area_list)
    mw11 = Dropdown(description='終点:', options=area_list)
    mw12 = SelectMultiple(description='日付:', options=df_all['timestamp_min_S'].dt.strftime('%Y-%m%-%d %H:%M:%S.%f').str[:10].unique(),
                         layout=Layout(height='500px'))
    mw13 = RadioButtons(description='Sum_or_Mean', options=['sum', 'mean'], disabled=False)

    # 比較用タブのWidgets  ※ cw = comparison_widgets
    comparison_list = ['経路', '曜日', '時刻', '日付', '午前午後', '平日休日', '平日祝日']
    cw0 = Label(value='①比較する要素の選択：')
    cw1 = SelectMultiple(description='フィルターの選択:',options=comparison_list, style=style) 
    cw2 = Label(value='②選択したフィルタから詳細をそれぞれ選ぶ')
    cw3 = Label(value='選択待ち...')
    cw4 = Label(value='③共通するフィルタから詳細を選ぶ：')
    cw5 = SelectMultiple(description='共通の経路:', options=direction_dict.keys(), layout=Layout(height='250px'))
    cw6 = SelectMultiple(description='共通の午前午後:', options=['午前', '午後'], layout=Layout(height='50px'))
    cw8 = SelectMultiple(description='共通の時刻:', options=np.sort(df_all['o\'clock'].str.zfill(2).unique()) )
    cw9 = SelectMultiple(description='共通の曜日:', options=df_all['day_of_week'].unique() )
    cw10 = Dropdown(description='共通の始点:', options=area_list)
    cw11 = Dropdown(description='共通の終点:', options=area_list)
    cw12 = SelectMultiple(description='共通の日付:', options=df_all['timestamp_min_S'].dt.strftime('%Y-%m%-%d %H:%M:%S.%f').str[:10].unique(),
                         layout=Layout(height='500px'))
    cw13 = RadioButtons(description='共通のSum_or_Mean', options=['mean', 'sum'], disabled=False)

   
    # 描画するキャンバスの設定
    img = mpimg.imread('daimasa_map2.jpg')
    #fig, ax = plt.subplots(ncols=2, figsize=(20,16))
    fig, ax = plt.subplots(2, 2, figsize=(20,32))
    plt.close()
    
    # 関数内関数------------------------------------------------------------------------------------------------------------
    
    # マップと矢印などを描画する関数
    def show_arrows3(path, first_area, last_area, Oclock, df, DayOfWeek, AMPM, Date, Sum_or_Mean,
                     path2, first_area2, last_area2, Oclock2, DayOfWeek2, AMPM2, Date2, Sum_or_Mean2,
                     co_path, co_first_area, co_last_area, co_Oclock, co_DayOfWeek, co_AMPM, co_Date, co_Sum_or_Mean,
                     savewidget):
        
        #複数の関数内で共通の変数
        nonlocal fig
        nonlocal ax
        #fig, ax = plt.subplots(ncols=2, figsize=(20,16))
        fig, ax = plt.subplots(2, 2, figsize=(20,32))
        ax[0,0] = plt.subplot2grid((4, 2), (0, 0), rowspan=3)
        ax[0,1] = plt.subplot2grid((4, 2), (0, 1), rowspan=3)
        #ax[1,0] = plt.subplot2grid((4, 2), (3, 0), colspan=2)
        
        # 比較用タブの描画設定----------------------------------------------------------------------------------------------
        if co_path:
            path = co_path
            path2 = co_path
        if co_first_area:
            first_area = co_first_area
            first_area2 = co_first_area
        if co_last_area:
            last_area = co_last_area
            last_area2 = co_last_area
        if co_Oclock:
            Oclock = co_Oclock
            Oclock2 = co_Oclock
        if co_DayOfWeek:
            DayOfWeek = co_DayOfWeek
            DayOfWeek2 = co_DayOfWeek
        if co_AMPM:
            AMPM = co_AMPM
            AMPM2 = co_AMPM
        if co_Date:    
            Date = co_Date
            Date2 = co_Date
        if co_Sum_or_Mean:
            Sum_or_Mean = co_Sum_or_Mean
            Sum_or_Mean2 = co_Sum_or_Mean

        # メイングラフ用の描画設定----------------------------------------------------------------------------------------------
        df_copy1 = df.copy()
        xlabel_text1 = []
        HOW_MANY_TIMES = 0
        
        if path:
            eng_path_pattern = '|'.join([direction_dict[k] for k in path])
            df_copy1 = df_copy1[df_copy1.Path.str.contains(eng_path_pattern)]
            #xlabel_text1.append(f'path={path[0]}:{path[-1]}')
        if Date:
            Date_pattern = '|'.join(Date)
            df_copy1 = df_copy1[df_copy1['timestamp_min_S'].dt.strftime('%Y-%m%-%d %H:%M:%S.%f').str.contains(Date_pattern)]
            xlabel_text1.append(f'Date={Date[0]}:{Date[-1]}')
        if Oclock:
            Oclock_pattern = '|'.join(Oclock)
            df_copy1 = df_copy1[df_copy1['o\'clock'].str.contains(Oclock_pattern)]
            xlabel_text1.append(f'Oclock={Oclock[0]}:{Oclock[-1]}')
        if DayOfWeek:
            DoW_pattern = '|'.join(DayOfWeek)
            df_copy1 = df_copy1[df_copy1['day_of_week'].str.contains(DoW_pattern)]
            xlabel_text1.append(f'day of week = {DayOfWeek}')
        
        if first_area:
            path = [k for k in direction_dict.keys() if k.startswith(first_area)]
            if last_area:
                path = set(path + [k for k in direction_dict.keys() if k.endswith(last_area)]) 
            eng_path_pattern = '|'.join([direction_dict[k] for k in path])
            df_copy1 = df_copy1[df_copy1.Path.str.contains(eng_path_pattern)]
            HOW_MANY_TIMES = 1
        elif last_area:
            path = [k for k in direction_dict.keys() if k.endswith(last_area)]
            eng_path_pattern = '|'.join([direction_dict[k] for k in path])
            df_copy1 = df_copy1[df_copy1.Path.str.contains(eng_path_pattern)]
            HOW_MANY_TIMES = 1

        if '午前' in AMPM:
            df_copy1 = df_copy1[(6 <= df_copy1['o\'clock'].astype(int)) & (df_copy1['o\'clock'].astype(int) < 12)]
            xlabel_text1.append(f'AMPM = AM')
        elif '午後' in AMPM :
            df_copy1 = df_copy1[(12 <= df_copy1['o\'clock'].astype(int)) & (df_copy1['o\'clock'].astype(int) < 18)]
            xlabel_text1.append(f'AMPM = PM')
        
        #テキストで表示する各経路の数値は、ココより下では変更されないので、平均の表示用に変数とする
        Path_counts = df_copy1.Path.value_counts()
        print('df_copy1.Path.value_counts()\n', Path_counts)
        print('path:', path)
        # otherを含まないバージョンの％表記
        Path_counts_per = df_copy1[~df_copy1.Path.str.contains('other')]['Path'].value_counts(normalize=True)*100
        if Sum_or_Mean == 'mean':
            Path_counts = round( Path_counts / df_copy1['timestamp_min_S'].dt.strftime('%Y-%m%-%d %H:%M:%S.%f').str[:10].nunique(), 1)
        
        # 描画処理用の関数設定
        #サイズの倍率は：3*(1 + HOW_MANY_TIMES*Path_counts[key]/Path_counts.min() )
        def arrow_and_zero_line(Path_counts, Path_counts_per, key, xy, dxy, color,P, HOW_MANY_TIMES=HOW_MANY_TIMES, n=0):
            ax[0,n].arrow(x=xy[0], y=xy[1], dx=dxy[0], dy=dxy[1], color=color, length_includes_head=True, 
                        width=3*(1 + HOW_MANY_TIMES * Path_counts[key] / Path_counts.min() ))
            try:
                ax[0,n].text(xy[0] +dxy[0] +P[0], xy[1] +dxy[1] +P[1], f'{Path_counts[key]}({round(Path_counts_per[key])}%)', color=color)
            except KeyError as error:
                print('error:',error)
                ax[0,n].text(xy[0] +dxy[0] +P[0], xy[1] +dxy[1] +P[1] , 'error', color='blue')
        
        def arrow_and_one_line(Path_counts, Path_counts_per, key, xy, dxy, color,P, HOW_MANY_TIMES=HOW_MANY_TIMES, n=0):
            ax[0,n].arrow(x=xy[0], y=xy[1], dx=dxy[0], dy=dxy[1], color=color, length_includes_head=True, 
                        width=3*(1 + HOW_MANY_TIMES * Path_counts[key] / Path_counts.min() ))
            ax[0,n].plot([xy[0]+P[2], xy[0]], [xy[1] +P[3], xy[1]], color=color,
                       linewidth=3*(1 + HOW_MANY_TIMES * Path_counts[key] / Path_counts.min() ))
            try:
                ax[0,n].text(xy[0] +dxy[0] +P[0], xy[1] +dxy[1] +P[1], f'{Path_counts[key]}({round(Path_counts_per[key])}%)', color=color)
            except KeyError as error:
                print('error:',error)
                ax[0,n].text(xy[0] +dxy[0] +P[0], xy[1] +dxy[1] +P[1] , 'error', color='blue')
                
        def arrow_and_two_lines(Path_counts, Path_counts_per, key, xy, dxy, color,P, HOW_MANY_TIMES=HOW_MANY_TIMES, n=0):
            ax[0,n].arrow(x=xy[0], y=xy[1], dx=dxy[0], dy=dxy[1], color=color, length_includes_head=True, 
                        width=3*(1 + HOW_MANY_TIMES * Path_counts[key] / Path_counts.min() ))
            ax[0,n].plot([xy[0]+P[2], xy[0]], [xy[1] +P[3], xy[1]], color=color,
                       linewidth=3*(1 + HOW_MANY_TIMES * Path_counts[key] / Path_counts.min() ))
            ax[0,n].plot([xy[0]+P[2]+P[4], xy[0]+P[5]], [xy[1]+P[3]+P[6], xy[1]+P[7]], color=color,
                       linewidth=3*(1 + HOW_MANY_TIMES * Path_counts[key] / Path_counts.min() ))
            try:
                ax[0,n].text(xy[0] +dxy[0] +P[0], xy[1] +dxy[1] +P[1], f'{Path_counts[key]}({round(Path_counts_per[key])}%)', color=color)
            except KeyError as error:
                print('error:',error)
                ax[0,n].text(xy[0] +dxy[0] +P[0], xy[1] +dxy[1] +P[1] , 'error', color='blue')
        
        if '本町3丁目～本町4丁目' in path:  
            arrow_and_zero_line(Path_counts, Path_counts_per, key='A1_to_A2', xy=(270,380), dxy=(-75,-300), color='blue',P=(-20,-30))
        if '本町4丁目～本町3丁目' in path:
            arrow_and_zero_line(Path_counts, Path_counts_per, key='A2_to_A1', xy=(225,100), dxy=(75*1.1, 300*1.1), color='red', P=(0,40) )
        if '行神橋～本町3丁目' in path:
            arrow_and_one_line(Path_counts, Path_counts_per, key='A3_to_A1', xy=(270,260), dxy=(30,130), color='green', P=(0,40,100,-16))
        if '本町3丁目～行神橋' in path:
            arrow_and_one_line(Path_counts, Path_counts_per, key='A1_to_A3', xy=(300,275), dxy=(100,-16), color='orange', P=(15,0,30,130))
        if '行神橋～本町4丁目' in path:
            arrow_and_one_line(Path_counts, Path_counts_per, key='A3_to_A2', xy=(300+10,230-10), dxy=(-30,-130), color='m', P=(-10,-15,120,-20))
        if '本町4丁目～行神橋' in path:
            arrow_and_one_line(Path_counts, Path_counts_per, key='A2_to_A3', xy=(285+15,245-15), dxy=(100,-16), color='k', P=(15,0,-40,-170))
        if '本町3丁目～市役所側' in path:
            arrow_and_one_line(Path_counts, Path_counts_per, key='A1_to_A4', xy=(195,240,), dxy=(-100,16), color='orange', P=(-40,30,40,150))
        if '市役所側～本町3丁目' in path:
            arrow_and_one_line(Path_counts, Path_counts_per, key='A4_to_A1', xy=(185,260), dxy=(40,150), color='green', P=(0,40,-100,16))
        if '行神橋～市役所側' in path:
            arrow_and_two_lines(Path_counts, Path_counts_per, key='A3_to_A4', xy=(240,220), dxy=(-130,20), color='purple', P=(-90,0,15,40,100,15,-16,40))
        if '市役所側～行神橋' in path:
            arrow_and_two_lines(Path_counts, Path_counts_per, key='A4_to_A3', xy=(260,250), dxy=(130,-20), color='cyan', P=(5,0,-15,-40,-100,-15,16,-40))
        if '本町4丁目～市役所側' in path:
            arrow_and_one_line(Path_counts, Path_counts_per, key='A2_to_A4', xy=(170,190), dxy=(-130,20), color='hotpink', P=(-5,-20,-30,-100))
        if '市役所側～本町4丁目' in path:
            arrow_and_one_line(Path_counts, Path_counts_per, key='A4_to_A2', xy=(185,205), dxy=(-30,-130), color='olive', P=(-60,-20,-100,15))
        """
        display(pd.crosstab(df_copy1['first_Area_S'], df_copy1['last_Area_E'], margins=True, normalize=True).style.format('{:.1%}')\
        .background_gradient(subset=pd.IndexSlice['A1':'other', 'A1':'other'], axis=0, cmap='YlGn', vmax=0.17))
        """

        """
        print('選択している経路：', path)
        print('選択している時間帯：', Oclock)   
        print('選択されている経路の数は', len(path))
        """

        # 比較グラフ用の描画設定----------------------------------------------------------------------------------------------
        df_copy2 = df.copy()
        xlabel_text2 = []
        HOW_MANY_TIMES2 = 0
        if path2:
            eng_path_pattern2 = '|'.join([direction_dict[k] for k in path2])
            df_copy2 = df_copy2[df_copy2.Path.str.contains(eng_path_pattern2)]
            #xlabel_text1.append(f'path={path[0]}:{path[-1]}')
        if Date2:
            Date_pattern2 = '|'.join(Date2)
            df_copy2 = df_copy2[df_copy2['timestamp_min_S'].dt.strftime('%Y-%m%-%d %H:%M:%S.%f').str.contains(Date_pattern2)]
            xlabel_text2.append(f'Date={Date2[0]}:{Date2[-1]}')
        if Oclock2:
            Oclock_pattern2 = '|'.join(Oclock2)
            df_copy2 = df_copy2[df_copy2['o\'clock'].str.contains(Oclock_pattern2)]
            xlabel_text2.append(f'Oclock={Oclock2[0]}:{Oclock2[-1]}')
        if DayOfWeek2:
            DoW_pattern2 = '|'.join(DayOfWeek2)
            df_copy2 = df_copy2[df_copy2['day_of_week'].str.contains(DoW_pattern2)]
            xlabel_text2.append(f'day of week = {DayOfWeek2}')
        if first_area2:
            path2 = [k for k in direction_dict.keys() if k.startswith(first_area2)]
            if last_area2:
                path2 = set(path2 + [k for k in direction_dict.keys() if k.endswith(last_area)]) 
            eng_path_pattern2 = '|'.join([direction_dict[k] for k in path2])
            df_copy2 = df_copy2[df_copy2.Path.str.contains(eng_path_pattern2)]
            HOW_MANY_TIMES2 = 1
        elif last_area2:
            path2 = [k for k in direction_dict.keys() if k.endswith(last_area2)]
            eng_path_pattern2 = '|'.join([direction_dict[k] for k in path2])
            df_copy2 = df_copy2[df_copy2.Path.str.contains(eng_path_pattern2)]
            HOW_MANY_TIMES2 = 1
        
        if '午前' in AMPM2:
            df_copy2 = df_copy2[(6 <= df_copy2['o\'clock'].astype(int)) & (df_copy2['o\'clock'].astype(int) < 12)]
            xlabel_text2.append(f'AMPM = AM')
        elif '午後' in AMPM2 :
            df_copy2 = df_copy2[(12 <= df_copy2['o\'clock'].astype(int)) & (df_copy2['o\'clock'].astype(int) < 18)]
            xlabel_text2.append(f'AMPM = PM')
        

        Path_counts2 = df_copy2.Path.value_counts()
        # otherを含まないバージョンの％表記
        Path_counts_per2 = df_copy2[~df_copy2.Path.str.contains('other')]['Path'].value_counts(normalize=True)*100
        if Sum_or_Mean2 == 'mean':
            Path_counts2 = round( Path_counts2 / df_copy2['timestamp_min_S'].dt.strftime('%Y-%m%-%d %H:%M:%S.%f').str[:10].nunique(), 1)

        # Subグラフの描画処理
        if '本町3丁目～本町4丁目' in path2:  
            arrow_and_zero_line(Path_counts=Path_counts2, Path_counts_per=Path_counts_per2, key='A1_to_A2', xy=(270,380), dxy=(-75,-300), color='blue',P=(-20,-30), HOW_MANY_TIMES=HOW_MANY_TIMES2, n=1)        
        if '本町4丁目～本町3丁目' in path2:
            arrow_and_zero_line(Path_counts=Path_counts2, Path_counts_per=Path_counts_per2, key='A2_to_A1', xy=(225,100), dxy=(75*1.1, 300*1.1), color='red', P=(0,40), HOW_MANY_TIMES=HOW_MANY_TIMES2, n=1 )
        if '行神橋～本町3丁目' in path2:
            arrow_and_one_line(Path_counts=Path_counts2, Path_counts_per=Path_counts_per2, key='A3_to_A1', xy=(270,260), dxy=(30,130), color='green', P=(0,40,100,-16), HOW_MANY_TIMES=HOW_MANY_TIMES2, n=1)
        if '本町3丁目～行神橋' in path2:
            arrow_and_one_line(Path_counts=Path_counts2, Path_counts_per=Path_counts_per2, key='A1_to_A3', xy=(300,275), dxy=(100,-16), color='orange', P=(15,0,30,130), HOW_MANY_TIMES=HOW_MANY_TIMES2, n=1)
        if '行神橋～本町4丁目' in path2:
            arrow_and_one_line(Path_counts=Path_counts2, Path_counts_per=Path_counts_per2, key='A3_to_A2', xy=(300,230), dxy=(-30,-130), color='m', P=(-10,-15,120,-20), HOW_MANY_TIMES=HOW_MANY_TIMES2, n=1)
        if '本町4丁目～行神橋' in path2:
            arrow_and_one_line(Path_counts=Path_counts2, Path_counts_per=Path_counts_per2, key='A2_to_A3', xy=(285,245), dxy=(100,-16), color='k', P=(15,0,-40,-170), HOW_MANY_TIMES=HOW_MANY_TIMES2, n=1)
        if '本町3丁目～市役所側' in path2:
            arrow_and_one_line(Path_counts=Path_counts2, Path_counts_per=Path_counts_per2, key='A1_to_A4', xy=(195,240,), dxy=(-100,16), color='orange', P=(-40,30,40,150), HOW_MANY_TIMES=HOW_MANY_TIMES2, n=1)
        if '市役所側～本町3丁目' in path2:
            arrow_and_one_line(Path_counts=Path_counts2, Path_counts_per=Path_counts_per2, key='A4_to_A1', xy=(185,260), dxy=(40,150), color='green', P=(0,40,-100,16), HOW_MANY_TIMES=HOW_MANY_TIMES2, n=1)           
        if '行神橋～市役所側' in path2:
            arrow_and_two_lines(Path_counts=Path_counts2, Path_counts_per=Path_counts_per2, key='A3_to_A4', xy=(240,220), dxy=(-130,20), color='purple', P=(-90,0,15,40,100,15,-16,40), HOW_MANY_TIMES=HOW_MANY_TIMES2, n=1)
        if '市役所側～行神橋' in path2:
            arrow_and_two_lines(Path_counts=Path_counts2, Path_counts_per=Path_counts_per2, key='A4_to_A3', xy=(260,250), dxy=(130,-20), color='cyan', P=(5,0,-15,-40,-100,-15,16,-40), HOW_MANY_TIMES=HOW_MANY_TIMES2, n=1)
        if '本町4丁目～市役所側' in path2:
            arrow_and_one_line(Path_counts=Path_counts2, Path_counts_per=Path_counts_per2, key='A2_to_A4', xy=(170,190), dxy=(-130,20), color='hotpink', P=(-5,-20,-30,-100), HOW_MANY_TIMES=HOW_MANY_TIMES2, n=1)
        if '市役所側～本町4丁目' in path2:
            arrow_and_one_line(Path_counts=Path_counts2, Path_counts_per=Path_counts_per2, key='A4_to_A2', xy=(185,205), dxy=(-30,-130), color='olive', P=(-60,-20,-100,15), HOW_MANY_TIMES=HOW_MANY_TIMES2, n=1)    
        """
        display(pd.crosstab(df_copy2['first_Area_S'], df_copy2['last_Area_E'], margins=True, normalize=True).style.format('{:.1%}')\
        .background_gradient(subset=pd.IndexSlice['A1':'other', 'A1':'other'], axis=0, cmap='YlGn', vmax=0.17))
        """
        
        # 同経路の比較のためのグラフ（ダッシュボード的な）
        def additional_graph():
            for path1_name in df_copy1.Path.unique():
                df1 = df_copy1[df_copy1.Path == path1_name].copy()
                df1['o\'clock'] = df1.loc[:,'o\'clock'].str.zfill(2)
                grouped1 = df1.groupby('o\'clock')
                ax[1,0].plot(grouped1['Path'].count() /\
                             df1['timestamp_min_S'].dt.strftime('%Y-%m%-%d %H:%M:%S.%f').str[:10].nunique(), label=f'main:{path1_name}')
                ax[1,0].legend()
            for path2_name in df_copy2.Path.unique():
                df2 = df_copy2[df_copy2.Path == path2_name].copy()
                df2['o\'clock'] = df2.loc[:,'o\'clock'].str.zfill(2)
                grouped2 = df2.groupby('o\'clock')
                ax[1,0].plot(grouped2['Path'].count() /\
                             df2['timestamp_min_S'].dt.strftime('%Y-%m%-%d %H:%M:%S.%f').str[:10].nunique(), label=f'sub:{path2_name}')
                ax[1,0].legend()
        print(f'len(df_copy1.Path.unique()):{len(df_copy1.Path.unique())}')
        print(f'len(df_copy2.Path.unique()):{len(df_copy2.Path.unique())}')
        if len(df_copy1.Path.unique()) + len(df_copy2.Path.unique()) <= 4:
            ax[1,0] = plt.subplot2grid((4, 2), (3, 0), colspan=2)
            additional_graph()

        # 実際にキャンバスへ描画する
        ax[0,0].set_xlim(0, 750)
        ax[0,0].set_ylim(550, 0)
        ax[0,0].set_title('Main')
        ax[0,1].set_xlim(0, 750)
        ax[0,1].set_ylim(550, 0)
        ax[0,1].set_title('Sub')
        
        ax[0,0].imshow(img, extent=[0, 640, 480, 0])
        ax[0,0].set_xlabel('\n'.join(xlabel_text1), size= 20)
        #ax[0].text(200, 30, '本町3丁目', color='blue')
        ax[0,1].imshow(img, extent=[0, 640, 480, 0])
        ax[0,1].set_xlabel('\n'.join(xlabel_text2), size= 20)
        
        #ax[1,1].close()
        
        #print('選択している経路：', path)
        #print('選択している時間帯：', Oclock2)  
        #print('選択している経路の数は', len(path2))
    
    
    
    #表示しているウィジェット自体をインタラクティブにする関数---------------------------------------------------------------
    def make_widgets_interactive(filters_list):
    
        nonlocal cw3
        nonlocal cw5
        nonlocal cw9
        nonlocal cw8
        nonlocal cw12
        nonlocal cw6
        nonlocal ui3
        nonlocal hbox3
        nonlocal children
        nonlocal tab
        print(f'選択されたフィルタは：{filters_list}')
        #cw5 = SelectMultiple(description='共通の経路:', options=direction_dict.keys(), layout=Layout(height='250px'))
        #cw6 = SelectMultiple(description='共通の午前午後:', options=['午前', '午後'], layout=Layout(height='50px'))
        #cw8 = SelectMultiple(description='共通の時刻:', options=np.sort(df_all['o\'clock'].str.zfill(2).unique()) )
        #cw9 = SelectMultiple(description='共通の曜日:', options=df_all['day_of_week'].unique() )
        #cw12 = SelectMultiple(description='共通の日付:', options=df_all['timestamp_min_S'].dt.strftime('%Y-%m%-%d %H:%M:%S.%f').str[:10].unique(),
         #            layout=Layout(height='500px'))

        if '曜日' in filters_list:
            cw3 = HBox([VBox([Label(value='＜メイングラフ用＞：'), w9]) , VBox([Label(value='＜サブグラフ用＞：'), mw9])])
            cw9 = Label(value='選択済み ==>曜日')
        elif '時刻' in filters_list:
            cw3 = HBox([VBox([Label(value='＜メイングラフ用＞：'), w8]) , VBox([Label(value='＜サブグラフ用＞：'), mw8])]) 
            cw8 = Label(value='選択済み ==>時刻')
        elif '日付' in filters_list:
            cw3 = HBox([VBox([Label(value='＜メイングラフ用＞：'), w12]) , VBox([Label(value='＜サブグラフ用＞：'), mw12])])
            cw12 = Label(value='選択済み ==>日付')
        elif '午前午後' in filters_list:
            cw3 = HBox([VBox([Label(value='＜メイングラフ用＞：'), w6]) , VBox([Label(value='＜サブグラフ用＞：'), mw6])])
            cw6 = Label(value='選択済み ==>午前午後')
        elif '経路' in filters_list:
            cw3 = HBox([VBox([Label(value='＜メイングラフ用＞：'), w5]) , VBox([Label(value='＜サブグラフ用＞：'), mw5])]) 
            cw5 = Label(value='選択済み ==>経路')
        else:
            print('まだ未実装')
        #print(f'中の階層での現在のcw3={cw3}')
        
        hbox3 = HBox([VBox([cw5, cw13]), VBox([cw8, cw10, cw11]), VBox([cw6, cw9, cw12])])
        ui3 = VBox([cw0, cw1, cw2, cw3, cw4, hbox3])
        children = [ui1, ui2, ui3]
        tab = Tab(children=children, _titles=titles)
        display(tab)
        #display(savewidget)
 
    
    
    # 描画したグラフを保存する関数------------------------------------------------------------------------------------
    def save_graph_as_image(b):
        nonlocal savewidget
        savewidget = 1
        #savewidget.value = 1
            
        #print('\n現在のsavewidget=', savewidget)
        if savewidget == 1:
            #print('savewidget=', savewidget)
            fig.savefig(f'経路分析用グラフ画像/{image_name.value}.png')
            print('画像を保存しました')
            print(f'ファイル名は{image_name.value}.png')
            savewidget = 0
    

    
    # グラフの保存用Widgets
    image_name = Text(description='保存する画像の名前を入力:', style=style)
    savewidget = Button(description='グラフの画像を保存する', style=style, layout=Layout(width='200px'))
    #save_output = Output()
    savewidget.value = 0
    savewidget.on_click(save_graph_as_image)
    
    # レイアウトの設定
    hbox1 = HBox([VBox([w5, w13]), VBox([w8, w10, w11]), VBox([w6, w9, w12])])
    hbox2 = HBox([VBox([mw5, mw13]), VBox([mw8, mw10, mw11]), VBox([mw6, mw9, mw12])])
    hbox3 = HBox([VBox([cw5, cw13]), VBox([cw8, cw10, cw11]), VBox([cw6, cw9, cw12])])
    ui1 = VBox([hbox1])
    ui2 = VBox([hbox2])
    ui3 = VBox([cw0, cw1, cw2, cw3, cw4])
    titles = {0:'メイングラフ用', 1:'サブグラフ用', 2:'比較用タブ'}
    children = [ui1, ui2, ui3]
    tab = Tab(children=children, _titles=titles)
    
    # インタラクティブ関数の設定
    graph_output = interactive_output(show_arrows3, {'path':w5, 'df':fixed(df_all),'first_area':w10, 'last_area':w11,'Oclock':w8, 'DayOfWeek':w9,
                                                'AMPM':w6,'Date':w12, 'Sum_or_Mean':w13,
                                                'path2':mw5, 'first_area2':mw10, 'last_area2':mw11,'Oclock2':mw8, 'DayOfWeek2':mw9,
                                                'AMPM2':mw6,'Date2':mw12, 'Sum_or_Mean2':mw13,
                                                'co_path':cw5,'co_first_area':cw10, 'co_last_area':cw11,'co_Oclock':cw8, 'co_DayOfWeek':cw9,
                                                'co_AMPM':cw6,'co_Date':cw12, 'co_Sum_or_Mean':cw13,
                                                'savewidget':savewidget})
    tab_output = interactive_output(make_widgets_interactive,{'filters_list':cw1})
    #save_output = interactive_output(save_graph_as_image, {'val':})
    
    

    # インタラクティブ部分の出力
    display(tab_output)
    display(graph_output)
    display(savewidget)
    display(image_name)

In [12]:
interactive_path_graph()

Output()

Output()

Button(description='グラフの画像を保存する', layout=Layout(width='200px'), style=ButtonStyle())

Text(value='', description='保存する画像の名前を入力:', style=DescriptionStyle(description_width='initial'))